Code for gathering model outputs into readable tsv files for their projection in Tensorboard Embedding projector: https://projector.tensorflow.org/ 


In [ ]:
!pip install transformers
!pip install torch.utils.tensorboard


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch.utils.tensorboard (from versions: none)
ERROR: No matching distribution found for torch.utils.tensorboard


In [ ]:
from transformers import AutoModel, RobertaModel
from torch.utils.tensorboard import SummaryWriter
from transformers.models.auto.tokenization_auto import AutoTokenizer
import pandas as pd
import re
import torch
import tensorflow as tf
import tensorboard as tb
import shutil
import os

tf.io.gfile = tb.compat.tensorflow_stub.io.gfile #torch and tensorflow compatibility

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_type = 'classification' #can choose between 'masked' or 'classification' objective
model_name = 'roberta-base'
checkpoint_folder = '/content/drive/MyDrive/model/' #The files needed are a folder with pytorch_model.bin and config.bin.
config_file = '/content/drive/MyDrive/model/config.json'
test_file = '/content/drive/MyDrive/CogALexV/test.tsv'
example_batch = slice(0,400)

if model_type == 'masked':
  special_token = tokenizer.mask_token
  if model_name.split('-')[0]=='roberta':
    special_token_id = 50264
  else:
    special_token_id = 103
else:
  special_token = tokenizer.sep_token
  if model_name.split('-')[0]=='roberta':
    special_token_id = 2
  else:
    special_token_id = 102

template = "' <W1> ' "+special_token+ " ' <W2> '."


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True, max_length=32)


In [ ]:
if model_name.split('-')[0]=='roberta':
  model = RobertaModel.from_pretrained(checkpoint_folder)
else:
  model = BertModel.from_pretrained(checkpoint_folder)

Some weights of the model checkpoint at /content/drive/MyDrive/T1_models_vis/ were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/T1_models_vis/ and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

verbalize test word-pairs

In [ ]:
df = pd.read_csv(test_file, sep='\t', names=['source', 'target', 'category'], header=None)
df['target'] = df['target'].str.split('/')
for i in range(len(df['target'])):
  df['target'].iloc[i] = df['target'].iloc[i][0]

In [ ]:
def verbalize(data, template):
  df = pd.DataFrame()  
  for i in range(len(data["source"])):
    w1=data['source'].iloc[i]
    w2= data['target'].iloc[i]
    verbalization = re.sub("<W1>", w1, template)
    verbalization = re.sub("<W2>", w2, verbalization)
    row = pd.DataFrame({'verb':[verbalization], 'correct':[w2], 'rel': [data["category"].iloc[i]], 'labels': [data["category"].iloc[i]]})
    df = pd.concat((df, row), axis = 0)
    df['labels'] = df['rel'].astype('category').cat.codes
  return(df)

verb_two_pairs_test = verbalize(df, template)

verb_two_pairs_test

,verb,correct,rel,labels
0,' coach ' </s> ' teach '.,teach,HYPER,1
0,' coach ' </s> ' teacher '.,teacher,HYPER,1
0,' coach ' </s> ' coat '.,coat,RANDOM,3
0,' coach ' </s> ' dog '.,dog,RANDOM,3
0,' coach ' </s> ' performance '.,performance,RANDOM,3
...,...,...,...,...
0,' rinse ' </s> ' clergyman '.,clergyman,RANDOM,3
0,' rinse ' </s> ' middle '.,middle,RANDOM,3
0,' rinse ' </s> ' nice '.,nice,RANDOM,3
0,' rinse ' </s> ' plain '.,plain,RANDOM,3


In [ ]:
inputs = tokenizer(verb_two_pairs_test['verb'].tolist()[example_batch], padding='max_length', max_length=32, return_tensors='pt')


In [ ]:
outputs = model(**inputs, output_hidden_states=True)

In [ ]:
layer=0
os.mkdir('/content/drive/MyDrive/results_vis')
while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists('/content/drive/MyDrive/results_vis/layer_' + str(layer)):
    os.mkdir('/content/drive/MyDrive/results_vis/layer_' + str(layer))
  
  example = 0
  tensors = []
  labels = []
  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in inputs['input_ids'][example]:
      if token != special_token_id:
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break
    example +=1 
    label = verb_two_pairs_test['verb'].iloc[example] + '\t' + verb_two_pairs_test['rel'].iloc[example]
    labels.append(label)
  
  path = '/content/drive/MyDrive/results_vis/layer_' + str(layer)
  
  writer=SummaryWriter(path)
  writer.add_embedding(torch.stack(tensors), metadata=labels)
  
  layer+=1

now you can upload metadata and tensors file into the embedding projector [API](https://projector.tensorflow.org/ ) of tensorflow